In [1]:
%pip install earthengine-api

import ee

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install geemap
import geemap
ee.Authenticate()
ee.Initialize()

Note: you may need to restart the kernel to use updated packages.



Successfully saved authorization token.


In [3]:
try:
  ee.Initialize()
  print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
  print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

Google Earth Engine has initialized successfully!


In [4]:
Map = geemap.Map(center=(48.406,11.690), zoom=11) # Set your study areacoordinates as center and change the zoom value for better visualization
Map

Map(center=[48.406, 11.69], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

In [34]:
roi = ee.FeatureCollection("users/gaoyu546/cropboun") # Read your Study area shape here from uploaded GEE Asset
aoi = roi.geometry()       # Convert Feature Collecion to Geometry as String

In [35]:
def landsat (image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0);
    saturationMask = image.select('QA_RADSAT').eq(0);
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True).updateMask(qaMask).updateMask(saturationMask);

In [36]:
dataset_l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
            .map(landsat)\
            .filterDate('2013-05-01', '2022-08-31')\
            .filterBounds(roi)\
            .filter(ee.Filter.calendarRange(121, 243, 'day_of_year'))
dataset = ee.ImageCollection("COPERNICUS/S2_SR")\
            .filterDate('2016-07-01', '2020-07-11')\
            .filterBounds(roi)\
            .filter('CLOUDY_PIXEL_PERCENTAGE <= 5')


In [37]:
listOfImages = dataset_l8.aggregate_array('system:index').getInfo()
print('Number of images in the collection: ', len(listOfImages))
listOfImages

Number of images in the collection:  269


['LC08_192026_20130515',
 'LC08_192026_20130616',
 'LC08_192026_20130702',
 'LC08_192026_20130718',
 'LC08_192026_20130803',
 'LC08_192026_20130819',
 'LC08_192026_20140502',
 'LC08_192026_20140603',
 'LC08_192026_20140619',
 'LC08_192026_20140705',
 'LC08_192026_20140721',
 'LC08_192026_20140806',
 'LC08_192026_20140822',
 'LC08_192026_20150505',
 'LC08_192026_20150606',
 'LC08_192026_20150622',
 'LC08_192026_20150708',
 'LC08_192026_20150724',
 'LC08_192026_20150809',
 'LC08_192026_20150825',
 'LC08_192026_20160507',
 'LC08_192026_20160523',
 'LC08_192026_20160608',
 'LC08_192026_20160624',
 'LC08_192026_20160710',
 'LC08_192026_20160726',
 'LC08_192026_20160811',
 'LC08_192026_20160827',
 'LC08_192026_20170510',
 'LC08_192026_20170526',
 'LC08_192026_20170611',
 'LC08_192026_20170627',
 'LC08_192026_20170713',
 'LC08_192026_20170729',
 'LC08_192026_20170814',
 'LC08_192026_20170830',
 'LC08_192026_20180513',
 'LC08_192026_20180529',
 'LC08_192026_20180630',
 'LC08_192026_20180716',


In [38]:
rgb_vis = {'min':6000, 'max':12000, 'bands':['B4','B3','B2']}
ndvi_colorizedVis = {"min":0,"max":1,"palette":["FFFFFF","CE7E45","DF923D", \
                   "F1B555","FCD163","99B718","74A901","66A000","529400","3E8601",\
                    "207401","056201","004C00","023B01","012E01","011D01","011301"]}
Map.addLayer(aoi)
comp = dataset_l8.median().select('SR_B5').clip(roi)

In [39]:
comp = dataset_l8.median().select('SR_B5').clip(roi)
for i in range(2013,2021):
    for j in range(5,9):
        ima=dataset_l8.filter(ee.Filter.calendarRange(j,j,'month'))\
        .filter(ee.Filter.calendarRange(i,i,'year'))
        med = ima.median().clip(roi)
        ndvi = med.normalizedDifference(['SR_B5', 'SR_B4']).rename(str(i)+'0'+str(j)+'ndvi').toDouble()
        evi = med.expression('2.5*((NIR-R)/(NIR+6*R+1-7.5*B))',{
                                        'NIR':med.select('SR_B5'),
                                        'R':med.select('SR_B4'),
                                        'B':med.select('SR_B2')}) \
                                        .rename((str(i)+'0'+str(j)+'evi')).toDouble()
        ndwi = med.normalizedDifference(['SR_B3', 'SR_B5']).rename(str(i)+'0'+str(j)+'ndwi').toDouble()
        lswi1 = med.normalizedDifference(['SR_B3', 'SR_B6']).rename(str(i)+'0'+str(j)+'lswi1').toDouble()
        lswi2 = med.normalizedDifference(['SR_B3', 'SR_B7']).rename(str(i)+'0'+str(j)+'lswi2').toDouble()
        B5=med.select('SR_B5').toUint16()
        texture = B5.glcmTexture()
        asm=texture.select('SR_B5_asm').rename(str(i)+'0'+str(j)+'asm').toDouble()
        corr=texture.select('SR_B5_corr').rename(str(i)+'0'+str(j)+'corr').toDouble()
        var=texture.select('SR_B5_var').rename(str(i)+'0'+str(j)+'var').toDouble()
        comp = comp.addBands(ndvi).addBands(evi).addBands(ndwi).addBands(lswi1).addBands(lswi2).addBands(asm).addBands(corr).addBands(var)

In [40]:
# Output Investigation
# Get information about the bands as a list. 
bandNames = comp.bandNames() 
print('Band Names: ', bandNames.getInfo()) # ee.List of band names 

Band Names:  ['SR_B5', '201305ndvi', '201305evi', '201305ndwi', '201305lswi1', '201305lswi2', '201305asm', '201305corr', '201305var', '201306ndvi', '201306evi', '201306ndwi', '201306lswi1', '201306lswi2', '201306asm', '201306corr', '201306var', '201307ndvi', '201307evi', '201307ndwi', '201307lswi1', '201307lswi2', '201307asm', '201307corr', '201307var', '201308ndvi', '201308evi', '201308ndwi', '201308lswi1', '201308lswi2', '201308asm', '201308corr', '201308var', '201405ndvi', '201405evi', '201405ndwi', '201405lswi1', '201405lswi2', '201405asm', '201405corr', '201405var', '201406ndvi', '201406evi', '201406ndwi', '201406lswi1', '201406lswi2', '201406asm', '201406corr', '201406var', '201407ndvi', '201407evi', '201407ndwi', '201407lswi1', '201407lswi2', '201407asm', '201407corr', '201407var', '201408ndvi', '201408evi', '201408ndwi', '201408lswi1', '201408lswi2', '201408asm', '201408corr', '201408var', '201505ndvi', '201505evi', '201505ndwi', '201505lswi1', '201505lswi2', '201505asm', '2015

In [49]:
# Get projection information from band 1. 
b2proj = comp.select('SR_B5').projection() 
print('Band 5 projection: ', b2proj.getInfo()) # ee.Projection object 

Band 5 projection:  {'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}


In [50]:
task = ee.batch.Export.image.toDrive(image = comp,description='comp13_20',scale=30,folder = 'crop',region=aoi)
task.start()

In [59]:
task.status()

{'state': <State.UNSUBMITTED: 'UNSUBMITTED'>}

In [52]:
for i in range(2013,2021):
    comp = dataset_l8.median().select('SR_B5').clip(roi)
    for j in range(5,9):
        ima=dataset_l8.filter(ee.Filter.calendarRange(j,j,'month'))\
        .filter(ee.Filter.calendarRange(i,i,'year'))
        med = ima.median().clip(roi)
        ndvi = med.normalizedDifference(['SR_B5', 'SR_B4']).rename('0'+str(j)+'ndvi').toDouble()
        evi = med.expression('2.5*((NIR-R)/(NIR+6*R+1-7.5*B))',{
                                        'NIR':med.select('SR_B5'),
                                        'R':med.select('SR_B4'),
                                        'B':med.select('SR_B2')}) \
                                        .rename(('0'+str(j)+'evi')).toDouble()
        ndwi = med.normalizedDifference(['SR_B3', 'SR_B5']).rename('0'+str(j)+'ndwi').toDouble()
        lswi1 = med.normalizedDifference(['SR_B3', 'SR_B6']).rename('0'+str(j)+'lswi1').toDouble()
        lswi2 = med.normalizedDifference(['SR_B3', 'SR_B7']).rename('0'+str(j)+'lswi2').toDouble()
        B5=med.select('SR_B5').toUint16()
        texture = B5.glcmTexture()
        asm=texture.select('SR_B5_asm').rename('0'+str(j)+'asm').toDouble()
        corr=texture.select('SR_B5_corr').rename('0'+str(j)+'corr').toDouble()
        var=texture.select('SR_B5_var').rename('0'+str(j)+'var').toDouble()
        comp = comp.addBands(ndvi).addBands(evi).addBands(ndwi).addBands(lswi1).addBands(lswi2).addBands(asm).addBands(corr).addBands(var)
        print(str(i)+'0'+str(j))
    task = ee.batch.Export.image.toDrive(image = comp,description=str(i)+'index',scale=30,folder = 'crop',region=aoi)
    task.start()
    print(str(i)+' is finished')

201305
201306
201307
201308
2013 is finished
201405
201406
201407
201408
2014 is finished
201505
201506
201507
201508
2015 is finished
201605
201606
201607
201608
2016 is finished
201705
201706
201707
201708
2017 is finished
201805
201806
201807
201808
2018 is finished
201905
201906
201907
201908
2019 is finished
202005
202006
202007
202008
2020 is finished


In [60]:
task.status()

{'state': <State.UNSUBMITTED: 'UNSUBMITTED'>}

In [62]:
gt_sample=ee.FeatureCollection("users/gaoyu546/sample_all")
samples=gt_sample

bands=['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'b10', 'b11', 'b12', 'b13', 'b14', 'b15', 'b16', 'b17', 'b18', 'b19', 'b20', 'b21', 'b22', 'b23', 'b24', 'b25', 'b26', 'b27', 'b28', 'b29', 'b30', 'b31', 'b32', 'b33']


In [56]:
withRandom = samples.randomColumn('random')
# We want to reserve some of the data for testing, to avoid overfitting the model.
split = 0.7;  # Roughly 70% training, 30% testing.
training = withRandom.filter(ee.Filter.lt('random', split))
testing = withRandom.filter(ee.Filter.gte('random', split))

In [64]:
# Train Random Forest (RF) classifier with default parameters.
trained = ee.Classifier.smileRandomForest(300).train(training, 'Class', bands)

# Classify the test FeatureCollection.
test = testing.classify(trained)

# Print the confusion matrix.
confusionMatrix = test.errorMatrix('Class', 'classification')
rf_accuracy = confusionMatrix.accuracy()
#print('Confusion Matrix', confusionMatrix)
#print('Validation overall accuracy: ', rf_accuracy)

# Classify the image with the same bands used for training.
for i in ['2013','2014','2015','2016','2017','2018']:
    data=ee.Image("users/gaoyu546/"+i+"index")
    result = data.select(bands).classify(trained)
    task = ee.batch.Export.image.toDrive(image = result,description=i+'class',scale=30,folder = 'crop',region=aoi)
    task.start()
# Display the clusters with random colors.
#Map.addLayer(result_rf4.randomVisualizer(), {}, 'RF_Scheme_4')
#Map

EEException: Collection.loadTable: Expected asset 'users/gaoyu546/sample_all' to be a Collection, found 'IndexedFolder'.

In [151]:
task.status()

{'state': 'COMPLETED',
 'description': '2018class',
 'creation_timestamp_ms': 1671555386618,
 'update_timestamp_ms': 1671555439701,
 'start_timestamp_ms': 1671555431890,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/#folders/1lmbNUZE66_48fNTi2h9CgEmQQ-8ogRsQ'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 1.8325011730194092,
 'id': 'EDKFARGIWKR7J663FEL6WCI2',
 'name': 'projects/earthengine-legacy/operations/EDKFARGIWKR7J663FEL6WCI2'}

In [153]:
Map.addLayer(result.randomVisualizer(), {}, 'RF_Scheme_4')
Map

Map(bottom=1451378.0, center=[48.39604337518057, 11.68125629425049], controls=(WidgetControl(options=['positio…